In [2]:
from linkedin_api import Linkedin
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm
import os

In [3]:
def convert_ts(urn):
    unix_ts = int(str(int(bin(int(urn))[2:43], 2))[:-3])
    
    return datetime.utcfromtimestamp(unix_ts).strftime('%Y-%m-%d %H:%M:%S')

In [10]:
def get_LinkedIn_posts(api, company, posts_nr):
    
    posts = api.get_company_updates(public_id=company)
    print(posts[-1])
    cols = ['urn', 'ts', 'company', 'numLikes', 'numComments', 'text']

    posts_data = pd.DataFrame(data=[], columns=cols)

    for i in tqdm(range(posts_nr)):
        # urn
        urn = posts[i]['urn'].split(':')[-1]

        # Timestamp
        ts = convert_ts(urn)

        # numLikes
        numLikes = posts[i]['value']['com.linkedin.voyager.feed.render.UpdateV2']['socialDetail']['totalSocialActivityCounts']['numLikes']

        # numComments
        numComments = posts[i]['value']['com.linkedin.voyager.feed.render.UpdateV2']['socialDetail']['totalSocialActivityCounts']['numComments']

        # Text
        text = posts[i]['value']['com.linkedin.voyager.feed.render.UpdateV2']['commentary']['text']['text']

        new_row = {'urn': urn,
                   'ts': ts,
                   'company': company,
                   'numLikes': numLikes,
                   'numComments': numComments,
                   'text': text}

        posts_data = posts_data.append(new_row, ignore_index=True)

    posts_data.set_index('urn', inplace=True)
    posts_data.sort_values(by='ts', ascending=True, inplace=True)
    
    if not os.path.isfile('posts.csv'):
        posts_data.to_csv('posts.csv', header='cols', encoding='utf-8')
    
    else: # else it exists so append without writing the header
        posts_data.to_csv('posts.csv', mode='a', header=False, encoding='utf-8')
        

In [5]:
def get_LinkedIn_comments(api, company, comments_nr):
    
    posts_data = pd.read_csv("posts.csv")
    posts_data = posts_data.loc[posts_data.company == company]

    cols = ['comment_urn', 'post_urn', 'ts', 'company', 'author', 'numLikes', 'lang', 'text']

    comments_data = pd.DataFrame(data=[], columns=cols)

    for post_urn in tqdm(posts_data.urn):

        post_urn = str(post_urn)

        # Get Comments
        try:
            comments = api.get_post_comments(post_urn, comment_count=comments_nr)
            n_comments = len(comments)
        except:
            n_comments = 0
            pass

        for j in range(n_comments):

            # urn
            comment_urn = comments[j]['urn'].split(',')[-1][:-1]

            # Timestamp
            ts = convert_ts(comment_urn)

            # Author
            try:
                author = comments[j]['commenter']['com.linkedin.voyager.feed.MemberActor']['urn'].split(':')[-1]
            except:
                author = comments[j]['commenter']['com.linkedin.voyager.feed.CompanyActor']['urn'].split(':')[-1]

            # numLikes
            numLikes = comments[j]['socialDetail']['likes']['paging']['total']

            try:
                lang = comments[j]['originalLanguage']
                
            except:
                lang = "Other"

            text = comments[j]['commentV2']['text']

            new_row = {'comment_urn': comment_urn,
                       'post_urn': post_urn,
                       'ts': ts,
                       'company': company,
                       'author': author,
                       'numLikes': numLikes,
                       'lang': lang,
                       'text': text}

            comments_data = comments_data.append(new_row, ignore_index=True)

    comments_data.set_index('comment_urn', inplace=True)
    
    if not os.path.isfile('comments.csv'):
        comments_data.to_csv('comments.csv', header='cols', encoding='utf-8')
    
    else: # else it exists so append without writing the header
        comments_data.to_csv('comments.csv', mode='a', header=False, encoding='utf-8')


In [12]:
api = Linkedin('info.holin@gmail.com', 'ganhamos_mesmo', refresh_cookies=False)

get_LinkedIn_posts(api, company, posts_nr)

{'urn': 'urn:li:activity:6853681455713837056', 'entityUrn': 'urn:li:fs_feedUpdate:(V2&COMPANY_FEED,urn:li:activity:6853681455713837056)', 'id': 'activity:6853681455713837056', 'permalink': 'https://www.linkedin.com/feed/update/urn:li:activity:6853681455713837056', 'tracking': {'requestId': 'b54d227a-c716-4b36-8de8-570d3eac514f', 'trackingId': 'esFgnLdPvir2hQCyPNEazw=='}, 'value': {'com.linkedin.voyager.feed.render.UpdateV2': {'actor': {'urn': 'urn:li:company:2851304', 'image': {'attributes': [{'miniCompany': {'objectUrn': 'urn:li:company:2851304', 'entityUrn': 'urn:li:fs_miniCompany:2851304', 'name': 'Sword Health', 'showcase': False, 'active': True, 'logo': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200, 'fileIdentifyingUrlPathSegment': '200_200/0/1656591808433?e=1673481600&v=beta&t=FHLtCdz3G4rjuvT6xChbUj_6E01YOX07Crqb4hSd2qc', 'expiresAt': 1673481600000, 'height': 200}, {'width': 100, 'fileIdentifyingUrlPathSegment': '100_100/0/1656591808433?e=1673481600&v=beta&t=y_

  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
posts_nr = 2
comments_nr = 5
company = "tesla-motors"

In [14]:
api = Linkedin('info.holin@gmail.com', 'ganhamos_mesmo', refresh_cookies=False)

get_LinkedIn_posts(api, company, posts_nr)

{'urn': 'urn:li:activity:6853681455713837056', 'entityUrn': 'urn:li:fs_feedUpdate:(V2&COMPANY_FEED,urn:li:activity:6853681455713837056)', 'id': 'activity:6853681455713837056', 'permalink': 'https://www.linkedin.com/feed/update/urn:li:activity:6853681455713837056', 'tracking': {'requestId': 'b54d227a-c716-4b36-8de8-570d3eac514f', 'trackingId': 'esFgnLdPvir2hQCyPNEazw=='}, 'value': {'com.linkedin.voyager.feed.render.UpdateV2': {'actor': {'urn': 'urn:li:company:2851304', 'image': {'attributes': [{'miniCompany': {'objectUrn': 'urn:li:company:2851304', 'entityUrn': 'urn:li:fs_miniCompany:2851304', 'name': 'Sword Health', 'showcase': False, 'active': True, 'logo': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200, 'fileIdentifyingUrlPathSegment': '200_200/0/1656591808433?e=1673481600&v=beta&t=FHLtCdz3G4rjuvT6xChbUj_6E01YOX07Crqb4hSd2qc', 'expiresAt': 1673481600000, 'height': 200}, {'width': 100, 'fileIdentifyingUrlPathSegment': '100_100/0/1656591808433?e=1673481600&v=beta&t=y_

  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
api.get_company("sword-health")

{'staffingCompany': False,
 'companyIndustries': [{'localizedName': 'Health, Wellness & Fitness',
   'entityUrn': 'urn:li:fs_industry:124'}],
 'callToAction': {'callToActionType': 'VIEW_WEBSITE',
  'visible': True,
  'callToActionMessage': {'textDirection': 'USER_LOCALE',
   'text': 'Visit website'},
  'url': 'https://swordhealth.com/?utm_source=linkedin&utm_medium=social&utm_campaign=custom_button'},
 'staffCount': 513,
 'adsRule': 'STANDARD',
 'companyEmployeesSearchPageUrl': 'https://www.linkedin.com/vsearch/p?f_CC=2851304',
 'viewerFollowingJobsUpdates': False,
 'staffCountRange': {'start': 201, 'end': 500},
 'permissions': {'landingPageAdmin': False,
  'admin': False,
  'adAccountHolder': False},
 'logo': {'image': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200,
      'fileIdentifyingUrlPathSegment': '200_200/0/1656591808433?e=1673481600&v=beta&t=FHLtCdz3G4rjuvT6xChbUj_6E01YOX07Crqb4hSd2qc',
      'expiresAt': 1673481600000,
      'height': 200},
     {'width': 1

In [19]:
api.get_company("tesla-motors")

{'staffingCompany': False,
 'companyIndustries': [{'localizedName': 'Automotive',
   'entityUrn': 'urn:li:fs_industry:53'}],
 'callToAction': {'callToActionType': 'VIEW_WEBSITE',
  'visible': True,
  'callToActionMessage': {'textDirection': 'USER_LOCALE',
   'text': 'Visit website'},
  'url': 'https://www.tesla.com/careers'},
 'staffCount': 59556,
 'adsRule': 'STANDARD',
 'companyEmployeesSearchPageUrl': 'https://www.linkedin.com/vsearch/p?f_CC=15564',
 'viewerFollowingJobsUpdates': False,
 'staffCountRange': {'start': 10001},
 'permissions': {'landingPageAdmin': False,
  'admin': False,
  'adAccountHolder': False},
 'logo': {'image': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200,
      'fileIdentifyingUrlPathSegment': '200_200/0/1607665771371?e=1673481600&v=beta&t=Rneziqb20X60sRtxJRFR9rWMqdZDRSKGj8-2iLMEALQ',
      'expiresAt': 1673481600000,
      'height': 200},
     {'width': 100,
      'fileIdentifyingUrlPathSegment': '100_100/0/1607665771371?e=1673481600&v=beta

In [20]:
api.get_company_updates(urn_id="2851304")

[{'urn': 'urn:li:activity:6736697001603403778',
  'entityUrn': 'urn:li:fs_feedUpdate:(V2&COMPANY_FEED,urn:li:activity:6736697001603403778)',
  'id': 'activity:6736697001603403778',
  'permalink': 'https://www.linkedin.com/feed/update/urn:li:activity:6736697001603403778',
  'tracking': {'requestId': 'dd4ad5b9-ba86-430a-bf30-49f145688f3f',
   'trackingId': 'vRvcstjY58cSFmglYMLTQQ=='},
  'value': {'com.linkedin.voyager.feed.render.UpdateV2': {'actor': {'urn': 'urn:li:company:15564',
     'image': {'attributes': [{'miniCompany': {'objectUrn': 'urn:li:company:15564',
         'entityUrn': 'urn:li:fs_miniCompany:15564',
         'name': 'Tesla',
         'showcase': False,
         'active': True,
         'logo': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200,
             'fileIdentifyingUrlPathSegment': '200_200/0/1607665771371?e=1673481600&v=beta&t=Rneziqb20X60sRtxJRFR9rWMqdZDRSKGj8-2iLMEALQ',
             'expiresAt': 1673481600000,
             'height': 200},
       

In [21]:
api.get_company_updates(urn_id="15564")

[{'urn': 'urn:li:activity:6736697001603403778',
  'entityUrn': 'urn:li:fs_feedUpdate:(V2&COMPANY_FEED,urn:li:activity:6736697001603403778)',
  'id': 'activity:6736697001603403778',
  'permalink': 'https://www.linkedin.com/feed/update/urn:li:activity:6736697001603403778',
  'tracking': {'requestId': 'dd4ad5b9-ba86-430a-bf30-49f145688f3f',
   'trackingId': 'vRvcstjY58cSFmglYMLTQQ=='},
  'value': {'com.linkedin.voyager.feed.render.UpdateV2': {'actor': {'urn': 'urn:li:company:15564',
     'image': {'attributes': [{'miniCompany': {'objectUrn': 'urn:li:company:15564',
         'entityUrn': 'urn:li:fs_miniCompany:15564',
         'name': 'Tesla',
         'showcase': False,
         'active': True,
         'logo': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200,
             'fileIdentifyingUrlPathSegment': '200_200/0/1607665771371?e=1673481600&v=beta&t=Rneziqb20X60sRtxJRFR9rWMqdZDRSKGj8-2iLMEALQ',
             'expiresAt': 1673481600000,
             'height': 200},
       